<a href="https://colab.research.google.com/github/kareemullah123456789/big_data_advanced/blob/main/2_spark_sql_tabular.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:

from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [1]:
!ls /content/drive/MyDrive/cde_data

 1342-0.txt
 broadcast_logs
 data.csv
 gutenberg_books
 prideandprejudice.csv
'pyngrok_UI_CODE_Working_with_RDDs_in_PySpark_(2).ipynb'
 pyspark_tutorial.ipynb
 sales_data.csv
 sample.csv
 sample_data.csv
 Section_2_Resilient_Distributed_Datasets_Transformations.ipynb
 Section_3_Resilient_Distributed_Datasets_Actions.ipynb
 Section_4_Spark_DataFrames_and_Transformations.ipynb
 simple_count.csv
 simple_count_single_partition.csv
 simple_count_single_partition_final.csv
 Spark_SQL.ipynb


In [2]:
from pyspark.sql import SparkSession
import pyspark.sql.functions as F
spark = SparkSession.builder.getOrCreate()

In [11]:
my_grocery_list = [
    ["Banana", 2, 1.74],
    ["Apple", 4, 2.04],
    ["Carrot", 1, 1.09],
    ["Cake", 1, 10.99],
 ]

In [12]:
df_grocery_list = spark.createDataFrame(
  my_grocery_list, ["Item", "Quantity", "Price"])

In [7]:
df_grocery_list.printSchema()

root
 |-- Item: string (nullable = true)
 |-- Quantity: long (nullable = true)
 |-- Price: double (nullable = true)



In [13]:
!ls /content/drive/MyDrive/cde_data/broadcast_logs

 BroadcastLogs_2018_Q3_M8_sample.CSV   Call_Signs.csv  'data dictionary.doc'   ReferenceTables


In [3]:
import os

DIRECTORY = "/content/drive/MyDrive/cde_data/broadcast_logs"
logs = spark.read.csv(
    os.path.join(DIRECTORY, "BroadcastLogs_2018_Q3_M8_sample.CSV"),
    sep="|",
    header=True,
    inferSchema=True,
    timestampFormat="yyyy-MM-dd", )

In [4]:
logs.show(5,False)

+--------------+------------+----------+----------+-------------------+----------------------+----------+---------------+-----------------+----------------+---------------+------------------+--------------+--------------------+------------+----------------+----------------+------------+------------+-------------------------------------------+----------------+--------+--------------------+------------------+----------------------+-------------+---------+---------+---------+---------+
|BroadcastLogID|LogServiceID|LogDate   |SequenceNO|AudienceTargetAgeID|AudienceTargetEthnicID|CategoryID|ClosedCaptionID|CountryOfOriginID|DubDramaCreditID|EthnicProgramID|ProductionSourceID|ProgramClassID|FilmClassificationID|ExhibitionID|Duration        |EndTime         |LogEntryDate|ProductionNO|ProgramTitle                               |StartTime       |Subtitle|NetworkAffiliationID|SpecialAttentionID|BroadcastOriginPointID|CompositionID|Producer1|Producer2|Language1|Language2|
+--------------+--------

In [5]:
logs.select("BroadcastLogID", "LogServiceID", "LogDate").show(5, False)

+--------------+------------+----------+
|BroadcastLogID|LogServiceID|LogDate   |
+--------------+------------+----------+
|1196192316    |3157        |2018-08-01|
|1196192317    |3157        |2018-08-01|
|1196192318    |3157        |2018-08-01|
|1196192319    |3157        |2018-08-01|
|1196192320    |3157        |2018-08-01|
+--------------+------------+----------+
only showing top 5 rows



In [ ]:

#Different ways of selecting the data

In [6]:
 # Using the string to column conversion
logs.select("BroadCastLogID", "LogServiceID", "LogDate")



DataFrame[BroadCastLogID: int, LogServiceID: int, LogDate: date]

In [7]:
 # Using the string to column conversion
logs.select(*["BroadCastLogID", "LogServiceID", "LogDate"])

DataFrame[BroadCastLogID: int, LogServiceID: int, LogDate: date]

In [8]:

# Passing the column object explicitly
logs.select(
    F.col("BroadCastLogID"), F.col("LogServiceID"), F.col("LogDate")
 )

DataFrame[BroadCastLogID: int, LogServiceID: int, LogDate: date]

In [9]:
logs.select(
    *[F.col("BroadCastLogID"), F.col("LogServiceID"), F.col("LogDate")]
 )

DataFrame[BroadCastLogID: int, LogServiceID: int, LogDate: date]

In [ ]:
#end

In [10]:
import numpy as np
column_split = np.array_split(np.array(logs.columns), len(logs.columns) // 3)
print(column_split)

[array(['BroadcastLogID', 'LogServiceID', 'LogDate'], dtype='<U22'), array(['SequenceNO', 'AudienceTargetAgeID', 'AudienceTargetEthnicID'],
      dtype='<U22'), array(['CategoryID', 'ClosedCaptionID', 'CountryOfOriginID'], dtype='<U22'), array(['DubDramaCreditID', 'EthnicProgramID', 'ProductionSourceID'],
      dtype='<U22'), array(['ProgramClassID', 'FilmClassificationID', 'ExhibitionID'],
      dtype='<U22'), array(['Duration', 'EndTime', 'LogEntryDate'], dtype='<U22'), array(['ProductionNO', 'ProgramTitle', 'StartTime'], dtype='<U22'), array(['Subtitle', 'NetworkAffiliationID', 'SpecialAttentionID'],
      dtype='<U22'), array(['BroadcastOriginPointID', 'CompositionID', 'Producer1'],
      dtype='<U22'), array(['Producer2', 'Language1', 'Language2'], dtype='<U22')]


In [11]:
#  for x in column_split:
#     logs.select(*x).show(5, False)

In [12]:
logs = logs.drop("BroadcastLogID", "SequenceNO")
 # Testing if we effectively got rid of the columns
print("BroadcastLogID" in logs.columns)  # => False
print("SequenceNo" in logs.columns)  # => False

False
False


In [13]:
logs = logs.select(
    *[x for x in logs.columns if x not in ["BroadcastLogID", "SequenceNO"]]
 )

In [14]:
logs.show(5,False)

+------------+----------+-------------------+----------------------+----------+---------------+-----------------+----------------+---------------+------------------+--------------+--------------------+------------+----------------+----------------+------------+------------+-------------------------------------------+----------------+--------+--------------------+------------------+----------------------+-------------+---------+---------+---------+---------+
|LogServiceID|LogDate   |AudienceTargetAgeID|AudienceTargetEthnicID|CategoryID|ClosedCaptionID|CountryOfOriginID|DubDramaCreditID|EthnicProgramID|ProductionSourceID|ProgramClassID|FilmClassificationID|ExhibitionID|Duration        |EndTime         |LogEntryDate|ProductionNO|ProgramTitle                               |StartTime       |Subtitle|NetworkAffiliationID|SpecialAttentionID|BroadcastOriginPointID|CompositionID|Producer1|Producer2|Language1|Language2|
+------------+----------+-------------------+----------------------+--------

In [86]:
# sample_frame.columns # => ['item', 'price', 'quantity', 'UPC']
# print(sample_frame.drop('item', 'UPC', 'prices').columns)

In [15]:
logs.select(F.col("Duration")).show(5)

+----------------+
|        Duration|
+----------------+
|02:00:00.0000000|
|00:00:30.0000000|
|00:00:15.0000000|
|00:00:15.0000000|
|00:00:15.0000000|
+----------------+
only showing top 5 rows



In [16]:
 print(logs.select(F.col("Duration")).dtypes)

[('Duration', 'string')]


In [17]:
logs.select(
    F.col("Duration"),
    F.col("Duration").substr(1, 2).cast("int").alias("dur_hours"),
    F.col("Duration").substr(4, 2).cast("int").alias("dur_minutes"),
    F.col("Duration").substr(7, 2).cast("int").alias("dur_seconds")
).distinct().show(5, False)  # jsut to compare things

+----------------+---------+-----------+-----------+
|Duration        |dur_hours|dur_minutes|dur_seconds|
+----------------+---------+-----------+-----------+
|00:04:52.0000000|0        |4          |52         |
|00:10:06.0000000|0        |10         |6          |
|00:26:41.0000000|0        |26         |41         |
|00:05:29.0000000|0        |5          |29         |
|00:08:18.0000000|0        |8          |18         |
+----------------+---------+-----------+-----------+
only showing top 5 rows



In [18]:
logs.select(
    F.col("Duration"),
    (
        F.col("Duration").substr(1, 2).cast("int") * 60 * 60 +  # Hours to seconds
        F.col("Duration").substr(4, 2).cast("int") * 60 +      # Minutes to seconds
        F.col("Duration").substr(7, 2).cast("int")             # Seconds
    ).alias("Duration_seconds")
).distinct().show(5, False)  # everything in seconds

+----------------+----------------+
|Duration        |Duration_seconds|
+----------------+----------------+
|01:59:30.0000000|7170            |
|00:31:00.0000000|1860            |
|00:28:08.0000000|1688            |
|00:10:30.0000000|630             |
|00:32:00.0000000|1920            |
+----------------+----------------+
only showing top 5 rows



In [19]:
logs.show(5,False)

+------------+----------+-------------------+----------------------+----------+---------------+-----------------+----------------+---------------+------------------+--------------+--------------------+------------+----------------+----------------+------------+------------+-------------------------------------------+----------------+--------+--------------------+------------------+----------------------+-------------+---------+---------+---------+---------+
|LogServiceID|LogDate   |AudienceTargetAgeID|AudienceTargetEthnicID|CategoryID|ClosedCaptionID|CountryOfOriginID|DubDramaCreditID|EthnicProgramID|ProductionSourceID|ProgramClassID|FilmClassificationID|ExhibitionID|Duration        |EndTime         |LogEntryDate|ProductionNO|ProgramTitle                               |StartTime       |Subtitle|NetworkAffiliationID|SpecialAttentionID|BroadcastOriginPointID|CompositionID|Producer1|Producer2|Language1|Language2|
+------------+----------+-------------------+----------------------+--------

In [20]:
# with column method
# same as abve but different method
logs = logs.withColumn(
    "Duration_seconds",
    (
        F.col("Duration").substr(1, 2).cast("int") * 60 * 60 +  # Hours to seconds
        F.col("Duration").substr(4, 2).cast("int") * 60 +      # Minutes to seconds
        F.col("Duration").substr(7, 2).cast("int")             # Seconds
    )
)

# Print schema and check results
logs.printSchema()
logs.show(5, False)


root
 |-- LogServiceID: integer (nullable = true)
 |-- LogDate: date (nullable = true)
 |-- AudienceTargetAgeID: integer (nullable = true)
 |-- AudienceTargetEthnicID: integer (nullable = true)
 |-- CategoryID: integer (nullable = true)
 |-- ClosedCaptionID: integer (nullable = true)
 |-- CountryOfOriginID: integer (nullable = true)
 |-- DubDramaCreditID: integer (nullable = true)
 |-- EthnicProgramID: integer (nullable = true)
 |-- ProductionSourceID: integer (nullable = true)
 |-- ProgramClassID: integer (nullable = true)
 |-- FilmClassificationID: integer (nullable = true)
 |-- ExhibitionID: integer (nullable = true)
 |-- Duration: string (nullable = true)
 |-- EndTime: string (nullable = true)
 |-- LogEntryDate: date (nullable = true)
 |-- ProductionNO: string (nullable = true)
 |-- ProgramTitle: string (nullable = true)
 |-- StartTime: string (nullable = true)
 |-- Subtitle: string (nullable = true)
 |-- NetworkAffiliationID: integer (nullable = true)
 |-- SpecialAttentionID: inte

In [ ]:
#Tidying our data frame: Renaming and reordering columns

In [21]:
# Check existing columns before renaming
print("Before renaming:", logs.columns)

# Rename column
logs = logs.withColumnRenamed("Duration_seconds", "duration_seconds")

# Check schema after renaming
logs.printSchema()
print("After renaming:", logs.columns)


Before renaming: ['LogServiceID', 'LogDate', 'AudienceTargetAgeID', 'AudienceTargetEthnicID', 'CategoryID', 'ClosedCaptionID', 'CountryOfOriginID', 'DubDramaCreditID', 'EthnicProgramID', 'ProductionSourceID', 'ProgramClassID', 'FilmClassificationID', 'ExhibitionID', 'Duration', 'EndTime', 'LogEntryDate', 'ProductionNO', 'ProgramTitle', 'StartTime', 'Subtitle', 'NetworkAffiliationID', 'SpecialAttentionID', 'BroadcastOriginPointID', 'CompositionID', 'Producer1', 'Producer2', 'Language1', 'Language2', 'Duration_seconds']
root
 |-- LogServiceID: integer (nullable = true)
 |-- LogDate: date (nullable = true)
 |-- AudienceTargetAgeID: integer (nullable = true)
 |-- AudienceTargetEthnicID: integer (nullable = true)
 |-- CategoryID: integer (nullable = true)
 |-- ClosedCaptionID: integer (nullable = true)
 |-- CountryOfOriginID: integer (nullable = true)
 |-- DubDramaCreditID: integer (nullable = true)
 |-- EthnicProgramID: integer (nullable = true)
 |-- ProductionSourceID: integer (nullable =

In [22]:
# Convert all column names to lowercase
logs = logs.toDF(*[x.lower() for x in logs.columns])

# Print schema to verify changes
logs.printSchema()


root
 |-- logserviceid: integer (nullable = true)
 |-- logdate: date (nullable = true)
 |-- audiencetargetageid: integer (nullable = true)
 |-- audiencetargetethnicid: integer (nullable = true)
 |-- categoryid: integer (nullable = true)
 |-- closedcaptionid: integer (nullable = true)
 |-- countryoforiginid: integer (nullable = true)
 |-- dubdramacreditid: integer (nullable = true)
 |-- ethnicprogramid: integer (nullable = true)
 |-- productionsourceid: integer (nullable = true)
 |-- programclassid: integer (nullable = true)
 |-- filmclassificationid: integer (nullable = true)
 |-- exhibitionid: integer (nullable = true)
 |-- duration: string (nullable = true)
 |-- endtime: string (nullable = true)
 |-- logentrydate: date (nullable = true)
 |-- productionno: string (nullable = true)
 |-- programtitle: string (nullable = true)
 |-- starttime: string (nullable = true)
 |-- subtitle: string (nullable = true)
 |-- networkaffiliationid: integer (nullable = true)
 |-- specialattentionid: inte

In [ ]:
# Selecting our columns in alphabetical order using select()

In [23]:



logs.select(sorted(logs.columns)).printSchema()

root
 |-- audiencetargetageid: integer (nullable = true)
 |-- audiencetargetethnicid: integer (nullable = true)
 |-- broadcastoriginpointid: integer (nullable = true)
 |-- categoryid: integer (nullable = true)
 |-- closedcaptionid: integer (nullable = true)
 |-- compositionid: integer (nullable = true)
 |-- countryoforiginid: integer (nullable = true)
 |-- dubdramacreditid: integer (nullable = true)
 |-- duration: string (nullable = true)
 |-- duration_seconds: integer (nullable = true)
 |-- endtime: string (nullable = true)
 |-- ethnicprogramid: integer (nullable = true)
 |-- exhibitionid: integer (nullable = true)
 |-- filmclassificationid: integer (nullable = true)
 |-- language1: integer (nullable = true)
 |-- language2: integer (nullable = true)
 |-- logdate: date (nullable = true)
 |-- logentrydate: date (nullable = true)
 |-- logserviceid: integer (nullable = true)
 |-- networkaffiliationid: integer (nullable = true)
 |-- producer1: string (nullable = true)
 |-- producer2: strin

In [24]:
#  for i in logs.columns:
#     logs.describe(i).show()

In [25]:
# for i in logs.columns:
#     logs.select(i).summary().show()

In [26]:
# Define the directory path
DIRECTORY = "/content/drive/MyDrive/cde_data/broadcast_logs"

# Read the CSV file into a DataFrame
log_identifier = spark.read.csv(
    os.path.join(DIRECTORY, "ReferenceTables/LogIdentifier.csv"),
    sep="|",       # Specify the delimiter
    header=True,   # Use the first row as column names
    inferSchema=True  # Automatically infer column data types
)

# Print schema to verify data types
log_identifier.printSchema()

# Show sample rows
log_identifier.show(5, truncate=False)

root
 |-- LogIdentifierID: string (nullable = true)
 |-- LogServiceID: integer (nullable = true)
 |-- PrimaryFG: integer (nullable = true)

+---------------+------------+---------+
|LogIdentifierID|LogServiceID|PrimaryFG|
+---------------+------------+---------+
|13ST           |3157        |1        |
|2000SM         |3466        |1        |
|70SM           |3883        |1        |
|80SM           |3590        |1        |
|90SM           |3470        |1        |
+---------------+------------+---------+
only showing top 5 rows



In [27]:
logs.join(
    log_identifier,
    on="LogServiceID",  # Common column to join on
    how="inner"         # Replace with desired join method
).show(5)


+------------+----------+-------------------+----------------------+----------+---------------+-----------------+----------------+---------------+------------------+--------------+--------------------+------------+----------------+----------------+------------+------------+--------------------+----------------+--------+--------------------+------------------+----------------------+-------------+---------+---------+---------+---------+----------------+---------------+---------+
|logserviceid|   logdate|audiencetargetageid|audiencetargetethnicid|categoryid|closedcaptionid|countryoforiginid|dubdramacreditid|ethnicprogramid|productionsourceid|programclassid|filmclassificationid|exhibitionid|        duration|         endtime|logentrydate|productionno|        programtitle|       starttime|subtitle|networkaffiliationid|specialattentionid|broadcastoriginpointid|compositionid|producer1|producer2|language1|language2|duration_seconds|LogIdentifierID|PrimaryFG|
+------------+----------+-----------

In [28]:
logs_and_channels_verbose = logs.join(
    log_identifier,  # Joining logs DataFrame with log_identifier DataFrame
    on="LogServiceID",  # Join based on the common column "LogServiceID"
    how="inner"  # Perform an inner join (keep only matching records)
)

logs_and_channels_verbose.show() # Print schema of the resulting DataFrame


+------------+----------+-------------------+----------------------+----------+---------------+-----------------+----------------+---------------+------------------+--------------+--------------------+------------+----------------+----------------+------------+------------+--------------------+----------------+--------+--------------------+------------------+----------------------+-------------+---------+---------+---------+---------+----------------+---------------+---------+
|logserviceid|   logdate|audiencetargetageid|audiencetargetethnicid|categoryid|closedcaptionid|countryoforiginid|dubdramacreditid|ethnicprogramid|productionsourceid|programclassid|filmclassificationid|exhibitionid|        duration|         endtime|logentrydate|productionno|        programtitle|       starttime|subtitle|networkaffiliationid|specialattentionid|broadcastoriginpointid|compositionid|producer1|producer2|language1|language2|duration_seconds|LogIdentifierID|PrimaryFG|
+------------+----------+-----------

In [29]:
log_identifier.show(5,False)

+---------------+------------+---------+
|LogIdentifierID|LogServiceID|PrimaryFG|
+---------------+------------+---------+
|13ST           |3157        |1        |
|2000SM         |3466        |1        |
|70SM           |3883        |1        |
|80SM           |3590        |1        |
|90SM           |3470        |1        |
+---------------+------------+---------+
only showing top 5 rows



In [30]:
logs_and_channels = logs.join(
    log_identifier,  # Joining logs DataFrame with log_identifier DataFrame
    "LogServiceID"  # Perform an equi-join on the "LogServiceID" column
)

logs_and_channels.show(5,False)  # Print schema of the resulting DataFrame


+------------+----------+-------------------+----------------------+----------+---------------+-----------------+----------------+---------------+------------------+--------------+--------------------+------------+----------------+----------------+------------+------------+------------------------------------------+----------------+--------+--------------------+------------------+----------------------+-------------+---------+---------+---------+---------+----------------+---------------+---------+
|logserviceid|logdate   |audiencetargetageid|audiencetargetethnicid|categoryid|closedcaptionid|countryoforiginid|dubdramacreditid|ethnicprogramid|productionsourceid|programclassid|filmclassificationid|exhibitionid|duration        |endtime         |logentrydate|productionno|programtitle                              |starttime       |subtitle|networkaffiliationid|specialattentionid|broadcastoriginpointid|compositionid|producer1|producer2|language1|language2|duration_seconds|LogIdentifierID|Pri

In [100]:
log_identifier.show(5,False)

+---------------+------------+---------+
|LogIdentifierID|LogServiceID|PrimaryFG|
+---------------+------------+---------+
|13ST           |3157        |1        |
|2000SM         |3466        |1        |
|70SM           |3883        |1        |
|80SM           |3590        |1        |
|90SM           |3470        |1        |
+---------------+------------+---------+
only showing top 5 rows



In [101]:
logs.show(5,False)

+------------+----------+-------------------+----------------------+----------+---------------+-----------------+----------------+---------------+------------------+--------------+--------------------+------------+----------------+----------------+------------+------------+-------------------------------------------+----------------+--------+--------------------+------------------+----------------------+-------------+---------+---------+---------+---------+----------------+
|logserviceid|logdate   |audiencetargetageid|audiencetargetethnicid|categoryid|closedcaptionid|countryoforiginid|dubdramacreditid|ethnicprogramid|productionsourceid|programclassid|filmclassificationid|exhibitionid|duration        |endtime         |logentrydate|productionno|programtitle                               |starttime       |subtitle|networkaffiliationid|specialattentionid|broadcastoriginpointid|compositionid|producer1|producer2|language1|language2|duration_seconds|
+------------+----------+-----------------

In [31]:
# Join the 'logs' DataFrame with 'log_identifier' on the condition that their 'LogServiceID' columns match
logs_and_channels_verbose = logs.join(
    log_identifier,
    logs["LogServiceID"] == log_identifier["LogServiceID"]
)

# Drop the 'LogServiceID' column from the 'log_identifier' DataFrame to avoid duplication
# Then select only the 'LogServiceID' column from the resulting DataFrame
logs_and_channels = logs_and_channels_verbose.drop(log_identifier["LogServiceID"]).select(
    "LogServiceID"
)

# Resulting DataFrame contains only the 'LogServiceID' column
# DataFrame[LogServiceID: int]

In [32]:
logs_and_channels_verbose.show(5,False)

+------------+----------+-------------------+----------------------+----------+---------------+-----------------+----------------+---------------+------------------+--------------+--------------------+------------+----------------+----------------+------------+------------+------------------------------------------+----------------+--------+--------------------+------------------+----------------------+-------------+---------+---------+---------+---------+----------------+---------------+------------+---------+
|logserviceid|logdate   |audiencetargetageid|audiencetargetethnicid|categoryid|closedcaptionid|countryoforiginid|dubdramacreditid|ethnicprogramid|productionsourceid|programclassid|filmclassificationid|exhibitionid|duration        |endtime         |logentrydate|productionno|programtitle                              |starttime       |subtitle|networkaffiliationid|specialattentionid|broadcastoriginpointid|compositionid|producer1|producer2|language1|language2|duration_seconds|LogIde

In [38]:
logs_and_channels.show(5,False)

+------------+
|LogServiceID|
+------------+
|3157        |
|3157        |
|3157        |
|3157        |
|3157        |
+------------+
only showing top 5 rows



In [54]:
!ls /content/drive/MyDrive/cde_data/broadcast_logs

 BroadcastLogs_2018_Q3_M8_sample.CSV   Call_Signs.csv  'data dictionary.doc'   ReferenceTables


In [55]:
!ls !ls /content/drive/MyDrive/cde_data/broadcast_logs/ReferenceTables

ls: cannot access '!ls': No such file or directory
/content/drive/MyDrive/cde_data/broadcast_logs/ReferenceTables:
BroadcastProducers.csv	     CD_ClosedCaption.csv    CD_FilmClassification.csv
CD_AirLanguage.csv	     CD_Composition.csv      CD_NetworkAffiliation.csv
CD_AudienceTargetAge.csv     CD_CountryOfOrigin.csv  CD_ProductionSource.csv
CD_AudienceTargetEthnic.csv  CD_DubDramaCredit.csv   CD_ProgramClass.csv
CD_BroadcastOriginPoint.csv  CD_EthnicProgram.csv    CD_SpecialAttention.csv
CD_Category.csv		     CD_Exhibition.csv	     LogIdentifier.csv


In [39]:
# Define the directory where the data files are stored
DIRECTORY = "/content/drive/MyDrive/cde_data/broadcast_logs"

# Load the CD_Category.csv file into a DataFrame
# Note: sep="|" specifies the delimiter, header=True indicates the first row contains column names.
cd_category = spark.read.csv(
    os.path.join(DIRECTORY, "ReferenceTables/CD_Category.csv"),
    sep="|",
    header=True,
    inferSchema=True,  # Automatically infer column data types
).select(
    "CategoryID",  # Select the CategoryID column
    "CategoryCD",  # Select the CategoryCD column
    F.col("EnglishDescription").alias("Category_Description")  # Rename EnglishDescription to Category_Description
)
cd_category.show(5,False)

+----------+----------+---------------------------+
|CategoryID|CategoryCD|Category_Description       |
+----------+----------+---------------------------+
|1         |010       |NEWS                       |
|2         |02        |CANREC  ANALYSIS (old)     |
|3         |02A       |ANALYSIS AND INTERPRETATION|
|4         |02B       |LONG-FORM DOCUMENTARY      |
|5         |030       |REPORTING & ACTUALITIES    |
+----------+----------+---------------------------+
only showing top 5 rows



In [40]:
# Load the CD_ProgramClass.csv file into a DataFrame
# Note: This file contains metadata about program classes.
cd_program_class = spark.read.csv(
    os.path.join(DIRECTORY, "ReferenceTables/CD_ProgramClass.csv"),
    sep="|",
    header=True,
    inferSchema=True,  # Automatically infer column data types
).select(
    "ProgramClassID",  # Select the ProgramClassID column
    "ProgramClassCD",  # Select the ProgramClassCD column
    F.col("EnglishDescription").alias("ProgramClass_Description")  # Rename EnglishDescription to ProgramClass_Description
)
cd_program_class.show(5,False)

+--------------+--------------+------------------------+
|ProgramClassID|ProgramClassCD|ProgramClass_Description|
+--------------+--------------+------------------------+
|1             |AUT           |AUTOPROMOTION           |
|2             |BAL           |BALANCE PROGRAMMING     |
|3             |COM           |COMMERCIAL MESSAGE      |
|4             |COR           |CORNERSTONE             |
|5             |DOC           |DOCUMENTARY             |
+--------------+--------------+------------------------+
only showing top 5 rows



In [41]:
logs_and_channels.printSchema()

root
 |-- LogServiceID: integer (nullable = true)



In [42]:
logs_and_channels_verbose.columns

['logserviceid',
 'logdate',
 'audiencetargetageid',
 'audiencetargetethnicid',
 'categoryid',
 'closedcaptionid',
 'countryoforiginid',
 'dubdramacreditid',
 'ethnicprogramid',
 'productionsourceid',
 'programclassid',
 'filmclassificationid',
 'exhibitionid',
 'duration',
 'endtime',
 'logentrydate',
 'productionno',
 'programtitle',
 'starttime',
 'subtitle',
 'networkaffiliationid',
 'specialattentionid',
 'broadcastoriginpointid',
 'compositionid',
 'producer1',
 'producer2',
 'language1',
 'language2',
 'duration_seconds',
 'LogIdentifierID',
 'LogServiceID',
 'PrimaryFG']

In [43]:
logs_and_channels_verbose = logs.alias("left").join(
    log_identifier.alias("right"),
    logs["LogServiceID"] == log_identifier["LogServiceID"],
 )
logs_and_channels_verbose.columns

['logserviceid',
 'logdate',
 'audiencetargetageid',
 'audiencetargetethnicid',
 'categoryid',
 'closedcaptionid',
 'countryoforiginid',
 'dubdramacreditid',
 'ethnicprogramid',
 'productionsourceid',
 'programclassid',
 'filmclassificationid',
 'exhibitionid',
 'duration',
 'endtime',
 'logentrydate',
 'productionno',
 'programtitle',
 'starttime',
 'subtitle',
 'networkaffiliationid',
 'specialattentionid',
 'broadcastoriginpointid',
 'compositionid',
 'producer1',
 'producer2',
 'language1',
 'language2',
 'duration_seconds',
 'LogIdentifierID',
 'LogServiceID',
 'PrimaryFG']

In [44]:
logs_and_channels_verbose.drop(F.col("right.LogServiceID")).select(
    "LogServiceID"
 )

DataFrame[LogServiceID: int]

In [45]:
logs_and_channels_verbose.columns

['logserviceid',
 'logdate',
 'audiencetargetageid',
 'audiencetargetethnicid',
 'categoryid',
 'closedcaptionid',
 'countryoforiginid',
 'dubdramacreditid',
 'ethnicprogramid',
 'productionsourceid',
 'programclassid',
 'filmclassificationid',
 'exhibitionid',
 'duration',
 'endtime',
 'logentrydate',
 'productionno',
 'programtitle',
 'starttime',
 'subtitle',
 'networkaffiliationid',
 'specialattentionid',
 'broadcastoriginpointid',
 'compositionid',
 'producer1',
 'producer2',
 'language1',
 'language2',
 'duration_seconds',
 'LogIdentifierID',
 'LogServiceID',
 'PrimaryFG']

In [47]:


# Join logs_and_channels with cd_category on the "CategoryID" column
# Note: A left join ensures all rows from logs_and_channels are retained.
full_log = logs_and_channels_verbose.join(cd_category, "CategoryID", how="left").join(
    cd_program_class, "ProgramClassID", how="left"  # Join with cd_program_class on "ProgramClassID"
)
full_log.show(5,False)


+--------------+----------+------------+----------+-------------------+----------------------+---------------+-----------------+----------------+---------------+------------------+--------------------+------------+----------------+----------------+------------+------------+------------------------------------------+----------------+--------+--------------------+------------------+----------------------+-------------+---------+---------+---------+---------+----------------+---------------+------------+---------+----------+------------------------------------------------+--------------+--------------------------------------+
|programclassid|categoryid|logserviceid|logdate   |audiencetargetageid|audiencetargetethnicid|closedcaptionid|countryoforiginid|dubdramacreditid|ethnicprogramid|productionsourceid|filmclassificationid|exhibitionid|duration        |endtime         |logentrydate|productionno|programtitle                              |starttime       |subtitle|networkaffiliationid|speci

In [48]:
DIRECTORY = "/content/drive/MyDrive/cde_data/broadcast_logs"
cd_category = spark.read.csv(
 os.path.join(DIRECTORY, "ReferenceTables/CD_Category.csv"),
 sep="|",
 header=True,
 inferSchema=True,
 ).select(
 "CategoryID",
 "CategoryCD",
 F.col("EnglishDescription").alias("Category_Description"),
)
cd_category.show(5,False)

+----------+----------+---------------------------+
|CategoryID|CategoryCD|Category_Description       |
+----------+----------+---------------------------+
|1         |010       |NEWS                       |
|2         |02        |CANREC  ANALYSIS (old)     |
|3         |02A       |ANALYSIS AND INTERPRETATION|
|4         |02B       |LONG-FORM DOCUMENTARY      |
|5         |030       |REPORTING & ACTUALITIES    |
+----------+----------+---------------------------+
only showing top 5 rows



In [49]:
cd_program_class = spark.read.csv(
 os.path.join(DIRECTORY, "ReferenceTables/CD_ProgramClass.csv"),
 sep="|",
 header=True,
 inferSchema=True,
 ).select(
 "ProgramClassID",
 "ProgramClassCD",
 F.col("EnglishDescription").alias("ProgramClass_Description"),
)
cd_program_class.show()

+--------------+--------------+------------------------+
|ProgramClassID|ProgramClassCD|ProgramClass_Description|
+--------------+--------------+------------------------+
|             1|          AUT |           AUTOPROMOTION|
|             2|          BAL |     BALANCE PROGRAMMING|
|             3|          COM |      COMMERCIAL MESSAGE|
|             4|          COR |             CORNERSTONE|
|             5|          DOC |             DOCUMENTARY|
|             6|          GIA |               GIVEAWAYS|
|             7|          ID  |    NETWORK IDENTIFIC...|
|             8|          INT |            INTERSTITIAL|
|             9|          LOC |       LOCAL ADVERTISING|
|            10|          MAG |        MAGAZINE PROGRAM|
|            11|          MER |           MERCHANDISING|
|            12|          MVC |        MUSIC VIDEO CLIP|
|            13|          NRN |    No recognized nat...|
|            14|          OFF |    SCHEDULED OFF AIR...|
|            15|          OFT |

In [123]:
logs_and_channels.show()

+------------+
|LogServiceID|
+------------+
|        3157|
|        3157|
|        3157|
|        3157|
|        3157|
|        3157|
|        3157|
|        3157|
|        3157|
|        3157|
|        3157|
|        3157|
|        3157|
|        3157|
|        3157|
|        3157|
|        3157|
|        3157|
|        3157|
|        3157|
+------------+
only showing top 20 rows



In [127]:
logs_and_channels = logs.join(log_identifier, "LogServiceID")
logs_and_channels.show(5,False)

+------------+----------+-------------------+----------------------+----------+---------------+-----------------+----------------+---------------+------------------+--------------+--------------------+------------+----------------+----------------+------------+------------+-------------------------------------------+----------------+--------+--------------------+------------------+----------------------+-------------+---------+---------+---------+---------+----------------+---------------+---------+
|logserviceid|logdate   |audiencetargetageid|audiencetargetethnicid|categoryid|closedcaptionid|countryoforiginid|dubdramacreditid|ethnicprogramid|productionsourceid|programclassid|filmclassificationid|exhibitionid|duration        |endtime         |logentrydate|productionno|programtitle                               |starttime       |subtitle|networkaffiliationid|specialattentionid|broadcastoriginpointid|compositionid|producer1|producer2|language1|language2|duration_seconds|LogIdentifierID|P

In [128]:
 full_log = logs_and_channels.join(cd_category, "CategoryID", how="left").join(
 cd_program_class, "ProgramClassID", how="left"
 )

In [129]:
full_log.show(5,False)

+--------------+----------+------------+----------+-------------------+----------------------+---------------+-----------------+----------------+---------------+------------------+--------------------+------------+----------------+----------------+------------+------------+-------------------------------------------+----------------+--------+--------------------+------------------+----------------------+-------------+---------+---------+---------+---------+----------------+---------------+---------+----------+------------------------------------------------+--------------+--------------------------------------+
|programclassid|categoryid|logserviceid|logdate   |audiencetargetageid|audiencetargetethnicid|closedcaptionid|countryoforiginid|dubdramacreditid|ethnicprogramid|productionsourceid|filmclassificationid|exhibitionid|duration        |endtime         |logentrydate|productionno|programtitle                               |starttime       |subtitle|networkaffiliationid|specialattention

In [130]:
(full_log
 .groupby("ProgramClassCD", "ProgramClass_Description")
 .agg(F.sum("duration_seconds").alias("duration_total"))
 .orderBy("duration_total", ascending=False).show(100, False)
 )

+--------------+--------------------------------------+--------------+
|ProgramClassCD|ProgramClass_Description              |duration_total|
+--------------+--------------------------------------+--------------+
|PGR           |PROGRAM                               |20992510      |
|COM           |COMMERCIAL MESSAGE                    |3519163       |
|PFS           |PROGRAM FIRST SEGMENT                 |1344762       |
|SEG           |SEGMENT OF A PROGRAM                  |1205998       |
|PRC           |PROMOTION OF UPCOMING CANADIAN PROGRAM|880600        |
|PGI           |PROGRAM INFOMERCIAL                   |679182        |
|PRO           |PROMOTION OF NON-CANADIAN PROGRAM     |335701        |
|OFF           |SCHEDULED OFF AIR TIME PERIOD         |142279        |
|ID            |NETWORK IDENTIFICATION MESSAGE        |74926         |
|NRN           |No recognized nationality             |59686         |
|MAG           |MAGAZINE PROGRAM                      |57622         |
|PSA  

In [50]:
full_log.groupby("ProgramClassCD", "ProgramClass_Description").agg(
    {"duration_seconds": "sum"}
 ).withColumnRenamed("sum(duration_seconds)", "duration_total").orderBy(
    "duration_total", ascending=False
 ).show(
    100, False
 )

+--------------+--------------------------------------+--------------+
|ProgramClassCD|ProgramClass_Description              |duration_total|
+--------------+--------------------------------------+--------------+
|PGR           |PROGRAM                               |29440180      |
|COM           |COMMERCIAL MESSAGE                    |4959005       |
|PFS           |PROGRAM FIRST SEGMENT                 |1897637       |
|SEG           |SEGMENT OF A PROGRAM                  |1535873       |
|PRC           |PROMOTION OF UPCOMING CANADIAN PROGRAM|1359433       |
|PGI           |PROGRAM INFOMERCIAL                   |765074        |
|PRO           |PROMOTION OF NON-CANADIAN PROGRAM     |416717        |
|OFF           |SCHEDULED OFF AIR TIME PERIOD         |187304        |
|ID            |NETWORK IDENTIFICATION MESSAGE        |117735        |
|MAG           |MAGAZINE PROGRAM                      |75624         |
|NRN           |No recognized nationality             |72195         |
|PSA  

In [54]:
logs.show(5,False)

+------------+----------+-------------------+----------------------+----------+---------------+-----------------+----------------+---------------+------------------+--------------+--------------------+------------+----------------+----------------+------------+------------+-------------------------------------------+----------------+--------+--------------------+------------------+----------------------+-------------+---------+---------+---------+---------+----------------+
|logserviceid|logdate   |audiencetargetageid|audiencetargetethnicid|categoryid|closedcaptionid|countryoforiginid|dubdramacreditid|ethnicprogramid|productionsourceid|programclassid|filmclassificationid|exhibitionid|duration        |endtime         |logentrydate|productionno|programtitle                               |starttime       |subtitle|networkaffiliationid|specialattentionid|broadcastoriginpointid|compositionid|producer1|producer2|language1|language2|duration_seconds|
+------------+----------+-----------------

In [55]:
logs.printSchema()

root
 |-- logserviceid: integer (nullable = true)
 |-- logdate: date (nullable = true)
 |-- audiencetargetageid: integer (nullable = true)
 |-- audiencetargetethnicid: integer (nullable = true)
 |-- categoryid: integer (nullable = true)
 |-- closedcaptionid: integer (nullable = true)
 |-- countryoforiginid: integer (nullable = true)
 |-- dubdramacreditid: integer (nullable = true)
 |-- ethnicprogramid: integer (nullable = true)
 |-- productionsourceid: integer (nullable = true)
 |-- programclassid: integer (nullable = true)
 |-- filmclassificationid: integer (nullable = true)
 |-- exhibitionid: integer (nullable = true)
 |-- duration: string (nullable = true)
 |-- endtime: string (nullable = true)
 |-- logentrydate: date (nullable = true)
 |-- productionno: string (nullable = true)
 |-- programtitle: string (nullable = true)
 |-- starttime: string (nullable = true)
 |-- subtitle: string (nullable = true)
 |-- networkaffiliationid: integer (nullable = true)
 |-- specialattentionid: inte

In [57]:
# Group the data by "ProgramClassCD" and "ProgramClass_Description"
# Note: These columns represent program categories and their descriptions.
full_log.groupby(
    "ProgramClassCD",
    "ProgramClass_Description"
).agg(
    # Aggregate the total duration (in seconds) for each group
    F.sum(F.col("Duration_seconds")).alias("total_duration_seconds")
).show(5,False)

+--------------+--------------------------------------+----------------------+
|ProgramClassCD|ProgramClass_Description              |total_duration_seconds|
+--------------+--------------------------------------+----------------------+
|PGR           |PROGRAM                               |29440180              |
|NRN           |No recognized nationality             |72195                 |
|PRC           |PROMOTION OF UPCOMING CANADIAN PROGRAM|1359433               |
|SO            |MAY IDENTIFY THE SIGN ON\OFF OF A DAY |59115                 |
|REG           |REGIONAL                              |6749                  |
+--------------+--------------------------------------+----------------------+
only showing top 5 rows



In [59]:
answer = (
    full_log.groupby("LogIdentifierID")
    .agg(
 F.sum(
            F.when(
                F.trim(F.col("ProgramClassCD")).isin(
                    ["COM", "PRC", "PGI", "PRO", "LOC", "SPO", "MER", "SOL"]
                ),
                F.col("duration_seconds"),
            ).otherwise(0)
        ).alias("duration_commercial"),
        F.sum("duration_seconds").alias("duration_total"),
    )
    .withColumn(
        "commercial_ratio", F.col(
            "duration_commercial") / F.col("duration_total")
    )
 )
answer.orderBy("commercial_ratio", ascending=False).show(1000, False)

+---------------+-------------------+--------------+---------------------+
|LogIdentifierID|duration_commercial|duration_total|commercial_ratio     |
+---------------+-------------------+--------------+---------------------+
|CIMT           |775                |775           |1.0                  |
|TLNSP          |15480              |15480         |1.0                  |
|TELENO         |17790              |17790         |1.0                  |
|MSET           |2700               |2700          |1.0                  |
|TRN            |13                 |13            |1.0                  |
|HPITV          |13                 |13            |1.0                  |
|TANG           |8125               |8125          |1.0                  |
|MUSIMAX        |23333              |23582         |0.9894410991434145   |
|MMAX           |23333              |23582         |0.9894410991434145   |
|MUSIP          |20587              |20912         |0.9844586840091814   |
|MPLU           |20587   

In [61]:
full_log.show(5,False)

+--------------+----------+------------+----------+-------------------+----------------------+---------------+-----------------+----------------+---------------+------------------+--------------------+------------+----------------+----------------+------------+------------+------------------------------------------+----------------+--------+--------------------+------------------+----------------------+-------------+---------+---------+---------+---------+----------------+---------------+------------+---------+----------+------------------------------------------------+--------------+--------------------------------------+
|programclassid|categoryid|logserviceid|logdate   |audiencetargetageid|audiencetargetethnicid|closedcaptionid|countryoforiginid|dubdramacreditid|ethnicprogramid|productionsourceid|filmclassificationid|exhibitionid|duration        |endtime         |logentrydate|productionno|programtitle                              |starttime       |subtitle|networkaffiliationid|speci

In [60]:
# Group the data by LogIdentifierID
answer = (
    full_log.groupby("LogIdentifierID")
    .agg(
        # Calculate the total duration of commercial-related broadcasts
        F.sum(
            F.when(
                F.trim(F.col("ProgramClassCD")).isin(  # Check if ProgramClassCD is in the list of commercial codes
                    ["COM", "PRC", "PGI", "PRO", "LOC", "SPO", "MER", "SOL"]
                ),
                F.col("duration_seconds"),  # Include the duration_seconds if the condition is met
            ).otherwise(0)  # Otherwise, include 0
        ).alias("duration_commercial"),  # Rename the resulting column to "duration_commercial"

        # Calculate the total duration of all broadcasts
        F.sum("duration_seconds").alias("duration_total")  # Sum up all durations and rename to "duration_total"
    )
    # Calculate the commercial ratio (commercial duration / total duration)
    .withColumn(
        "commercial_ratio",
        F.col("duration_commercial") / F.col("duration_total")  # Divide commercial duration by total duration
    )
)

# Sort the results by commercial_ratio in descending order and display the top 1000 rows
answer.orderBy("commercial_ratio", ascending=False).show(1000, False)

+---------------+-------------------+--------------+---------------------+
|LogIdentifierID|duration_commercial|duration_total|commercial_ratio     |
+---------------+-------------------+--------------+---------------------+
|CIMT           |775                |775           |1.0                  |
|TLNSP          |15480              |15480         |1.0                  |
|TELENO         |17790              |17790         |1.0                  |
|MSET           |2700               |2700          |1.0                  |
|TRN            |13                 |13            |1.0                  |
|HPITV          |13                 |13            |1.0                  |
|TANG           |8125               |8125          |1.0                  |
|MUSIMAX        |23333              |23582         |0.9894410991434145   |
|MMAX           |23333              |23582         |0.9894410991434145   |
|MUSIP          |20587              |20912         |0.9844586840091814   |
|MPLU           |20587   

In [62]:
answer_no_null = answer.dropna(subset=["commercial_ratio"])

In [63]:
answer_no_null.orderBy(
    "commercial_ratio", ascending=False).show(1000, False)

+---------------+-------------------+--------------+---------------------+
|LogIdentifierID|duration_commercial|duration_total|commercial_ratio     |
+---------------+-------------------+--------------+---------------------+
|CIMT           |775                |775           |1.0                  |
|TLNSP          |15480              |15480         |1.0                  |
|TELENO         |17790              |17790         |1.0                  |
|MSET           |2700               |2700          |1.0                  |
|TRN            |13                 |13            |1.0                  |
|HPITV          |13                 |13            |1.0                  |
|TANG           |8125               |8125          |1.0                  |
|MUSIMAX        |23333              |23582         |0.9894410991434145   |
|MMAX           |23333              |23582         |0.9894410991434145   |
|MUSIP          |20587              |20912         |0.9844586840091814   |
|MPLU           |20587   

In [64]:
# Replace null values in the DataFrame with 0
answer_no_null = answer.fillna(0)

# Sort the results by commercial_ratio in descending order and display the top 1000 rows
answer_no_null.orderBy(
    "commercial_ratio", ascending=False
).show(1000, False)

+---------------+-------------------+--------------+---------------------+
|LogIdentifierID|duration_commercial|duration_total|commercial_ratio     |
+---------------+-------------------+--------------+---------------------+
|CIMT           |775                |775           |1.0                  |
|TLNSP          |15480              |15480         |1.0                  |
|TELENO         |17790              |17790         |1.0                  |
|MSET           |2700               |2700          |1.0                  |
|TRN            |13                 |13            |1.0                  |
|HPITV          |13                 |13            |1.0                  |
|TANG           |8125               |8125          |1.0                  |
|MUSIMAX        |23333              |23582         |0.9894410991434145   |
|MMAX           |23333              |23582         |0.9894410991434145   |
|MUSIP          |20587              |20912         |0.9844586840091814   |
|MPLU           |20587   

In [105]:
# # Import necessary libraries
# import os
# from pyspark.sql import SparkSession
# import pyspark.sql.functions as F

# # Initialize Spark session
# # Note: The appName describes the purpose of this Spark job.
# spark = SparkSession.builder.appName(
#     "Compute Commercial Ratios for TV Channels"
# ).getOrCreate()

# # Set log level to WARN to reduce unnecessary output
# spark.sparkContext.setLogLevel("WARN")

# # Define the directory where the data is stored
# DIRECTORY = "/content/drive/MyDrive/cde_data/broadcast_logs"

# # Load broadcast logs dataset
# # Note: sep="|" specifies the delimiter, header=True indicates the first row contains column names.
# logs = spark.read.csv(
#     os.path.join(DIRECTORY, "BroadcastLogs_2018_Q3_M8_sample.CSV"),
#     sep="|",
#     header=True,
#     inferSchema=True,
# )

# # Drop unnecessary columns from the logs DataFrame
# # Note: Dropping "BroadcastLogID" and "SequenceNO" simplifies the dataset.
# logs = logs.drop("BroadcastLogID", "SequenceNO")

# # Convert Duration from HH:MM:SS format to seconds
# # Note: substr() extracts parts of the string, and cast() converts them to integers.
# logs = logs.withColumn(
#     "duration_seconds",
#     (
#         F.col("Duration").substr(1, 2).cast("int") * 60 * 60  # Hours to seconds
#         + F.col("Duration").substr(4, 2).cast("int") * 60     # Minutes to seconds
#         + F.col("Duration").substr(7, 2).cast("int")          # Seconds
#     ),
# )



In [106]:
# # Load log identifier dataset
# # Note: Filtering PrimaryFG == 1 ensures we only use primary logs.
# log_identifier = spark.read.csv(
#     os.path.join(DIRECTORY, "ReferenceTables/LogIdentifier.csv"),
#     sep="|",
#     header=True,
#     inferSchema=True,
# ).where(F.col("PrimaryFG") == 1)

# log_identifier.printSchema()

+---------------+-------------------+--------------+----------------+
|LogIdentifierID|duration_commercial|duration_total|commercial_ratio|
+---------------+-------------------+--------------+----------------+
|           CJCO|                  0|          NULL|             0.0|
|          BRAVO|                  0|          NULL|             0.0|
|           CFTF|                  0|          NULL|             0.0|
|           CKCS|                  0|          NULL|             0.0|
|           CJNT|                  0|          NULL|             0.0|
|           CKES|                  0|          NULL|             0.0|
|           CHBX|                  0|          NULL|             0.0|
|         BBCKID|                  0|          NULL|             0.0|
|           BOOK|                  0|          NULL|             0.0|
|           CHAN|                  0|          NULL|             0.0|
|         CEVASI|                  0|          NULL|             0.0|
|            CMT|   

+---------------+-------------------+--------------+----------------+
|LogIdentifierID|duration_commercial|duration_total|commercial_ratio|
+---------------+-------------------+--------------+----------------+
|CJCO           |0                  |NULL          |0.0             |
|BRAVO          |0                  |NULL          |0.0             |
|CFTF           |0                  |NULL          |0.0             |
|CKCS           |0                  |NULL          |0.0             |
|CJNT           |0                  |NULL          |0.0             |
|CKES           |0                  |NULL          |0.0             |
|CHBX           |0                  |NULL          |0.0             |
|BBCKID         |0                  |NULL          |0.0             |
|BOOK           |0                  |NULL          |0.0             |
|CHAN           |0                  |NULL          |0.0             |
|CEVASI         |0                  |NULL          |0.0             |
|CMT            |0  